<a href="https://colab.research.google.com/github/mdzafri/openvino-workshop/blob/main/Perceptron/4_1_MNIST_with_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Prepare Dataset

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Load the dataset
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# OPTIONAL: Display some dataset samples as an image 
%matplotlib inline
import matplotlib.pyplot as plt
import random

ROWS = 6
random_indices = random.sample(range(x_train.shape[0]), ROWS*ROWS)
sample_images = x_train[random_indices, :]
plt.clf()

fig, axes = plt.subplots(ROWS,ROWS, 
                         figsize=(ROWS,ROWS),
                         sharex=True, sharey=True) 

for i in range(ROWS*ROWS):
    subplot_row = i//ROWS 
    subplot_col = i%ROWS
    ax = axes[subplot_row, subplot_col]

    plottable_image = np.reshape(sample_images[i,:], (28,28))
    ax.imshow(plottable_image, cmap='gray_r')
    
    ax.set_xbound([0,28])

plt.tight_layout()
plt.show()

In [ ]:
# Normalize images to the [0, 1] range
# This is to make the calculations more efficient
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class label vectors to binary class matrices (convert to 1-hot format)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## (Optional) Use TensorBoard to display some nice graphs

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import datetime
# create keras TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# specify the log directory
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# 2. Select/Design Model

In [ ]:
# set up the model structure
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Flatten(),
        # layers.Dense(128, activation='relu'),  # hidden layer
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

# 3. Train the model

In [ ]:
# set the loss, optimizer and metrics
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# train/fit the model
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1)

In [ ]:
# Evaluate the trained model performance
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

## (Optional) Perform training for viewing data in Tensorboard

In [ ]:
# train the model and save training performance parameters into training_history
training_history = model.fit(
    x_train, # input data 
    y_train, # output classes
    batch_size=128, 
    epochs=10, 
    validation_split=0.1, 
    verbose=0, # Suppress chatty output; use Tensorboard instead 
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

In [ ]:
tensorboard --logdir logs/fit

# 4. Test the trained model

In [ ]:
# Make a canvas for user to draw a digit
# then save the drawing as a png file
# source: https://gist.github.com/korakot/8409b3feec20f159d8a50b0a811d3bca
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

canvas_html = """
<canvas width=%d height=%d></canvas>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.fillStyle = "white";
ctx.fillRect(0, 0, canvas.width, canvas.height);
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(filename='drawing.png', w=150, h=150, line_width=10):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
    print("image saved as: ")
    print(filename)
  # return len(binary)

draw()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageOps # import pillow image manipulation tool

# Load the image to be tested
user_image = Image.open('drawing.png')
user_image = ImageOps.grayscale(user_image)
user_image = ImageOps.invert(user_image)

# Resize to input_shape
user_image = user_image.resize((input_shape[0],input_shape[1]))
plt.imshow(user_image)
user_image = np.array(user_image).astype("float32") / 255
# user_image = np.expand_dims(user_image, axis=0)
user_image = user_image.reshape(-1, 28, 28, 1)
# print("user_image shape:", user_image.shape)

# Predict the class of the drawing 
result = model.predict(user_image)
print(result)
result = np.argmax(result,axis=1)
print("The AI thinks this is the number:", result[0])
